In [68]:
import pandas as pd

In [69]:
df = pd.read_csv("spotify_millsongdata.csv")

In [70]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [71]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [72]:
df.shape

(57650, 4)

In [73]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [74]:
df =df.sample(2000).drop('link', axis=1).reset_index(drop=True)

In [75]:
df.head(10)

,artist,song,text
0,Phil Collins,Sussudio,There's this girl that's been on my mind \r\n...
1,Little Mix,They Just Don't Know You,(JADE) \r\nDaddy doesn't think \r\nThat you'...
2,Cheap Trick,Can't Stop Fallin' Into Love,"\r\nHey, little ladies, there's some cool yo..."
3,Fleetwood Mac,Make Me A Mask,Make me a mask \r\nThat I can wear \r\n'Till...
4,Bob Marley,All In One,Bend down low \r\nLet me tell you what I know...
5,Queen,My Melancholy Blues,Another party's over \r\nAnd I'm left cold so...
6,Elvis Costello,Chemistry Class,She throws back her hair [hands] and she shows...
7,Notorious B.I.G.,Player's Anthem,"Niggas uh, bitches ha uh \r\n \r\n(Niggas) G..."
8,Ozzy Osbourne,Fire In The Sky,The introduction to his heartache began as a c...
9,Frank Zappa,Beauty Knows No Pain,"Frank zappa (lead guitar, vocals) \r\nIke wil..."


In [76]:
df['text'][0]

"There's this girl that's been on my mind  \r\nAll the time, Sussudio oh oh  \r\nNow she don't even know my name  \r\nBut I think she likes me just the same  \r\nSussudio oh oh  \r\n  \r\nOh if she called me I'd be there  \r\nI'd come running anywhere  \r\nShe's all I need, all my life  \r\nI feel so good if I just say the word  \r\nSussudio, just say the word  \r\nOh Sussudio  \r\n  \r\nNow I know that I'm too young  \r\nMy love has just begun  \r\nSussudio oh oh  \r\nOoh give me a chance, give me a sign  \r\nI'll show her anytime  \r\nSussudio oh oh  \r\n  \r\nAh, I've just got to have her, have her now  \r\nI've got to get closer but I don't know how  \r\nShe makes me nervous and makes me scared  \r\nBut I feel so good if I just say the word  \r\nSussudio just say the word  \r\nOh Sussudio, oh  \r\n  \r\nAh, she's all I need all of my life  \r\nI feel so good if I just say the word  \r\nSussudio I just say the word  \r\nOh Sussudio I just say the word  \r\nOh Sussudio I'll say the w

In [77]:
df.shape



(2000, 3)


Text Cleaning/ Text Preprocessing

Text Preprocessing

In [80]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [81]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [82]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [84]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [85]:
similarity[0]

array([1.        , 0.1265217 , 0.05573158, ..., 0.04146024, 0.03087858,
       0.06720691])

In [89]:
df[df['song'] == 'Sussudio']

,artist,song,text
0,Phil Collins,Sussudio,there 's thi girl that 's been on my mind all ...


In [87]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:10]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [91]:
recommendation('Sussudio')

['OMG',
 'Got To Be Certain',
 'Dr. Phibes Rises Again',
 'Oh Oh I Love Her So',
 'Oh Me, Oh My Sweet Baby',
 'Moses',
 'Moving Pictures',
 'Listen To The Music',
 'Lock Me Up']

In [93]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))